In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Load RetinaNet model

In [12]:
model = keras.models.load_model('snapshots/resnet50_csv_06.h5', custom_objects=custom_objects)
#print(model.summary())

/home/huray/venvs/ml/lib/python3.5/site-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Initialize data generators

In [16]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

# # create a generator for testing data
# val_generator = CocoGenerator(
#     '/srv/datasets/COCO',
#     'val2017',
#     val_image_data_generator,
#     batch_size=1,
# )
# create a generator for testing data
val_generator = CSVGenerator(
    '/home/huray/data/CBIS-DDSM/img/data_test.csv',
    '/home/huray/data/CBIS-DDSM/img/class.csv',
    val_image_data_generator,
    batch_size=1
)

f = open('/home/huray/data/CBIS-DDSM/img/data_test.csv', 'r')
num_of_test_files = len(f.readlines())
print(num_of_test_files)
f.close()

606


## Run detection on example

In [18]:
for index in range(num_of_test_files):
    flag = False
    # load image
    image = val_generator.load_image(index)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    annotations = val_generator.load_annotations(index)
    index += 1

    # process image
    start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
    

    # visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
#         if score > 0: print(score)
#         if score < 0.5:
#             continue
#         else:
#             flag = True
        if score < 0.01:
            continue
        else:
            print(score)
            flag = True

        b = detections[0, idx, :4].astype(int)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 3)
#         print(b)
#         if b[0] > 0 and b[1] > 0 and b[2] < 1700 and b[3] < 2100:
#             cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 3)
#             flag = True
#         else:
#             continue
        caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

    # visualize annotations
    for annotation in annotations:
        label = int(annotation[4])
        b = annotation[:4].astype(int)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
        caption = "{}".format(val_generator.label_to_name(label))
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

    if flag is True:
        plt.figure(figsize=(15, 15))
        plt.axis('off')
        plt.imshow(draw)
        plt.show()
        
#     plt.figure(figsize=(15, 15))
#     plt.axis('off')
#     plt.imshow(draw)
#     plt.show()